In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# FID Score for RGB images

In [ ]:
import numpy as np
import cv2
import os
import PIL

instances1 = []
# Set the path for real images (google drive location) 
dir_path = 'drive/MyDrive/Tiles dataset/im_1'
# Load in the images
for file_name in os.listdir(dir_path):
    image = PIL.Image.open(dir_path+'/'+file_name)
    image_array = np.array(image)
    instances1.append(image_array)

instances2 = []
# Set the path for generated/fake images (google drive location) 
dir_path = 'drive/MyDrive/Tiles dataset/im_2'
# Load in the images
for file_name in os.listdir(dir_path):
    image = PIL.Image.open(dir_path+'/'+file_name)
    image_array = np.array(image)
    instances2.append(image_array)

instances1 = np.array(instances1)
instances2 = np.array(instances2)

In [ ]:
import numpy
from numpy import cov
from numpy import trace
from numpy import iscomplexobj
from numpy import asarray
from numpy.random import randint
from scipy.linalg import sqrtm
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input
from keras.datasets.mnist import load_data
from skimage.transform import resize
 
# scale an array of images to a new size
def scale_images(images, new_shape):
	images_list = list()
	for image in images:
		# resize with nearest neighbor interpolation
		new_image = resize(image, new_shape, 0)
		# store
		images_list.append(new_image)
	return asarray(images_list)
 
# calculate frechet inception distance
def calculate_fid(model, images1, images2):
	# calculate activations
	act1 = model.predict(images1)
	act2 = model.predict(images2)
	# calculate mean and covariance statistics
	mu1, sigma1 = act1.mean(axis=0), cov(act1, rowvar=False)
	mu2, sigma2 = act2.mean(axis=0), cov(act2, rowvar=False)
	# calculate sum squared difference between means
	ssdiff = numpy.sum((mu1 - mu2)**2.0)
	# calculate sqrt of product between cov
	covmean = sqrtm(sigma1.dot(sigma2))
	# check and correct imaginary numbers from sqrt
	if iscomplexobj(covmean):
		covmean = covmean.real
	# calculate score
	fid = ssdiff + trace(sigma1 + sigma2 - 2.0 * covmean)
	return fid
 
# prepare the inception v3 model
model = InceptionV3(include_top=False, pooling='avg', input_shape=(299,299,3))
# define two fake collections of images
images1 = instances1
images2 = instances2
print('Prepared', images1.shape, images2.shape)
# convert integer to floating point values
images1 = images1.astype('float32')
images2 = images2.astype('float32')
# resize images
images1 = scale_images(images1, (299,299,3))
images2 = scale_images(images2, (299,299,3))
print('Scaled', images1.shape, images2.shape)
# pre-process images
images1 = preprocess_input(images1)
images2 = preprocess_input(images2)
# fid between images1 and images1
fid = calculate_fid(model, images1, images2)
print('FID Score: %.3f' % fid)